In [1]:
#import important libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing             import LabelEncoder
from sklearn.preprocessing             import StandardScaler
from sklearn.linear_model              import LogisticRegression
from sklearn.model_selection           import KFold, StratifiedKFold, train_test_split
from sklearn.metrics                   import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, precision_score, recall_score, precision_recall_curve
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
#read the data file
df = pd.read_csv('nyc_taxi_trip_duration.csv')

In [3]:
#view the data head
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,400
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,1100
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,1635
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,1141
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,848


In [4]:
df[['pickup_datetime','dropoff_datetime']] = df[['pickup_datetime','dropoff_datetime']].apply(lambda _: pd.to_datetime(_,format='%Y-%m-%d %H:%M:%S.%f', errors='coerce'))

In [5]:
new_df = pd.DataFrame({"pKyear": df['pickup_datetime'].dt.year,
              "Pkmonth": df['pickup_datetime'].dt.month,
              "pKday": df['pickup_datetime'].dt.day,
              "pKhour": df['pickup_datetime'].dt.hour,
              "pKdayofyear": df['pickup_datetime'].dt.dayofyear,
              "pKweek": df['pickup_datetime'].dt.week,
              "pKdayofweek": df['pickup_datetime'].dt.dayofweek,
              "pKdayofweekname": df['pickup_datetime'].dt.day_name(),
              "pKquarter": df['pickup_datetime'].dt.quarter,
              
              "dFyear": df['dropoff_datetime'].dt.year,
              "dFmonth": df['dropoff_datetime'].dt.month,
              "dFday": df['dropoff_datetime'].dt.day,
              "dFhour": df['dropoff_datetime'].dt.hour,
              "dFdayofyear": df['dropoff_datetime'].dt.dayofyear,
              "dFweek": df['dropoff_datetime'].dt.week,
              "dFdayofweek": df['dropoff_datetime'].dt.dayofweek,
              "dFdayofweekname": df['dropoff_datetime'].dt.day_name(),
              "dFquarter": df['dropoff_datetime'].dt.quarter,
             })
new_df.head()

,pKyear,Pkmonth,pKday,pKhour,pKdayofyear,pKweek,pKdayofweek,pKdayofweekname,pKquarter,dFyear,dFmonth,dFday,dFhour,dFdayofyear,dFweek,dFdayofweek,dFdayofweekname,dFquarter
0,2016,2,29,16,60,9,0,Monday,1,2016,2,29,16,60,9,0,Monday,1
1,2016,3,11,23,71,10,4,Friday,1,2016,3,11,23,71,10,4,Friday,1
2,2016,2,21,17,52,7,6,Sunday,1,2016,2,21,18,52,7,6,Sunday,1
3,2016,1,5,9,5,1,1,Tuesday,1,2016,1,5,10,5,1,1,Tuesday,1
4,2016,2,17,6,48,7,2,Wednesday,1,2016,2,17,6,48,7,2,Wednesday,1


In [6]:
complete_data = pd.concat([df, new_df], axis=1)
complete_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,pKquarter,dFyear,dFmonth,dFday,dFhour,dFdayofyear,dFweek,dFdayofweek,dFdayofweekname,dFquarter
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,...,1,2016,2,29,16,60,9,0,Monday,1
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,...,1,2016,3,11,23,71,10,4,Friday,1
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,...,1,2016,2,21,18,52,7,6,Sunday,1
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,...,1,2016,1,5,10,5,1,1,Tuesday,1
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,...,1,2016,2,17,6,48,7,2,Wednesday,1


In [7]:
complete_data['duration'] = (complete_data['dropoff_datetime'][0] - complete_data['pickup_datetime'][0])

In [8]:
complete_data['duration'] = complete_data.apply(lambda x: (x['dropoff_datetime'] - x['pickup_datetime']).seconds, axis=1)

In [9]:
complete_data[['duration','trip_duration']]

,duration,trip_duration
0,400,400
1,1100,1100
2,1635,1635
3,1141,1141
4,848,848
...,...,...
729317,296,296
729318,315,315
729319,673,673
729320,447,447


In [27]:
#check the data types
complete_data.dtypes

vendor_id               int64
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
trip_duration           int64
pKyear                  int64
Pkmonth                 int64
pKday                   int64
pKhour                  int64
pKdayofyear             int64
pKweek                  int64
pKdayofweek             int64
pKdayofweekname        object
pKquarter               int64
dFyear                  int64
dFmonth                 int64
dFday                   int64
dFhour                  int64
dFdayofyear             int64
dFweek                  int64
dFdayofweek             int64
dFquarter               int64
duration                int64
is_weekday              int64
haversine_distance    float64
direction             float64
dtype: object

In [11]:
complete_data['is_weekday']=0 

for i in range(0, len(complete_data)):
    if ((complete_data['pKdayofweek'][i] == 5) | (complete_data['pKdayofweek'][i] == 6)):
        complete_data['is_weekday'][i] = 0
    else: 
        complete_data['is_weekday'][i] = 1

<ipython-input-11-e2951d63fafa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_data['is_weekday'][i] = 1
<ipython-input-11-e2951d63fafa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_data['is_weekday'][i] = 0


In [12]:
complete_data['is_weekday']

0         1
1         1
2         0
3         1
4         1
         ..
729317    0
729318    1
729319    1
729320    0
729321    1
Name: is_weekday, Length: 729322, dtype: int64

In [13]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def direction_array(lat1, lng1, lat2, lng2):
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))


complete_data['haversine_distance'] = haversine_array(complete_data['pickup_latitude'].values, 
                                                     complete_data['pickup_longitude'].values, 
                                                     complete_data['dropoff_latitude'].values, 
                                                     complete_data['dropoff_longitude'].values)


complete_data['direction'] = direction_array(complete_data['pickup_latitude'].values, 
                                          complete_data['pickup_longitude'].values, 
                                          complete_data['dropoff_latitude'].values, 
                                          complete_data['dropoff_longitude'].values)


In [14]:
complete_data =complete_data.drop(['id','pickup_datetime','dropoff_datetime','store_and_fwd_flag','dFdayofweekname'], axis=1)
complete_data

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,pKyear,Pkmonth,pKday,...,dFday,dFhour,dFdayofyear,dFweek,dFdayofweek,dFquarter,duration,is_weekday,haversine_distance,direction
0,2,1,-73.953918,40.778873,-73.963875,40.771164,400,2016,2,29,...,29,16,60,9,0,1,400,1,1.199073,-135.634530
1,1,2,-73.988312,40.731743,-73.994751,40.694931,1100,2016,3,11,...,11,23,71,10,4,1,1100,1,4.129111,-172.445217
2,2,2,-73.997314,40.721458,-73.948029,40.774918,1635,2016,2,21,...,21,18,52,7,6,1,1635,0,7.250753,34.916093
3,2,6,-73.961670,40.759720,-73.956779,40.780628,1141,2016,1,5,...,5,10,5,1,1,1,1141,1,2.361097,10.043567
4,1,1,-74.017120,40.708469,-73.988182,40.740631,848,2016,2,17,...,17,6,48,7,2,1,848,1,4.328534,34.280582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729317,2,2,-73.965919,40.789780,-73.952637,40.789181,296,2016,5,21,...,21,13,142,20,5,2,296,0,1.120223,93.403814
729318,1,1,-73.996666,40.737434,-74.001320,40.731911,315,2016,2,22,...,22,0,53,8,0,1,315,1,0.728705,-147.443222
729319,1,1,-73.997849,40.761696,-74.001488,40.741207,673,2016,4,15,...,15,19,106,15,4,2,673,1,2.298776,-172.335340
729320,1,1,-74.006706,40.708244,-74.013550,40.713814,447,2016,6,19,...,19,9,171,24,6,2,447,0,0.846316,-42.964076


### Scaling Numerical Features for Logistic Regression

Now, we remember that there are a lot of outliers in the dataset especially when it comes to previous and current balance features. Also, the distributions are skewed for these features if you recall from the EDA. We will take 2 steps to deal with that here:
* Log Transformation
* Standard Scaler

Standard scaling is anyways a necessity when it comes to linear models and we have done that here after doing log transformation on all balance features.

In [15]:
np.sum(pd.isnull(complete_data))

vendor_id             0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
trip_duration         0
pKyear                0
Pkmonth               0
pKday                 0
pKhour                0
pKdayofyear           0
pKweek                0
pKdayofweek           0
pKdayofweekname       0
pKquarter             0
dFyear                0
dFmonth               0
dFday                 0
dFhour                0
dFdayofyear           0
dFweek                0
dFdayofweek           0
dFquarter             0
duration              0
is_weekday            0
haversine_distance    0
direction             0
dtype: int64

In [16]:
complete_data.vendor_id.value_counts()

2    390481
1    338841
Name: vendor_id, dtype: int64

In [17]:
complete_data['vendor_id'] = complete_data.apply(lambda x: (x['vendor_id'] - 1), axis=1)

In [18]:
complete_data

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,pKyear,Pkmonth,pKday,...,dFday,dFhour,dFdayofyear,dFweek,dFdayofweek,dFquarter,duration,is_weekday,haversine_distance,direction
0,1,1,-73.953918,40.778873,-73.963875,40.771164,400,2016,2,29,...,29,16,60,9,0,1,400,1,1.199073,-135.634530
1,0,2,-73.988312,40.731743,-73.994751,40.694931,1100,2016,3,11,...,11,23,71,10,4,1,1100,1,4.129111,-172.445217
2,1,2,-73.997314,40.721458,-73.948029,40.774918,1635,2016,2,21,...,21,18,52,7,6,1,1635,0,7.250753,34.916093
3,1,6,-73.961670,40.759720,-73.956779,40.780628,1141,2016,1,5,...,5,10,5,1,1,1,1141,1,2.361097,10.043567
4,0,1,-74.017120,40.708469,-73.988182,40.740631,848,2016,2,17,...,17,6,48,7,2,1,848,1,4.328534,34.280582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729317,1,2,-73.965919,40.789780,-73.952637,40.789181,296,2016,5,21,...,21,13,142,20,5,2,296,0,1.120223,93.403814
729318,0,1,-73.996666,40.737434,-74.001320,40.731911,315,2016,2,22,...,22,0,53,8,0,1,315,1,0.728705,-147.443222
729319,0,1,-73.997849,40.761696,-74.001488,40.741207,673,2016,4,15,...,15,19,106,15,4,2,673,1,2.298776,-172.335340
729320,0,1,-74.006706,40.708244,-74.013550,40.713814,447,2016,6,19,...,19,9,171,24,6,2,447,0,0.846316,-42.964076


In [29]:
X = complete_data.drop(['duration', 'pKdayofweekname'], axis =1)
Y = complete_data['duration']

In [30]:
# Splitting the data into Train and Validation set
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=1/3, random_state=1)

In [ ]:
model = LogisticRegression()
model.fit(xtrain,ytrain)


In [ ]:
pred = model.predict(xtest)

### AUC ROC Curve & Confusion Matrix 

Now, let us quickly look at the AUC-ROC curve for our logistic regression model and also the confusion matrix to see where the logistic regression model is failing here.

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(ytest,pred) 
auc = roc_auc_score(ytest, pred) 
plt.figure(figsize=(12,8)) 
plt.plot(fpr,tpr,label="Validation AUC-ROC="+str(auc)) 
x = np.linspace(0, 1, 1000)
plt.plot(x, x, linestyle='-')
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.legend(loc=4) 
plt.show()

In [ ]:
#recall score
recall_score(ytest,pred)

In [ ]:
def cv_score(ml_model, rstate = 12, thres = 0.5, cols = df.columns):
    i = 1
    cv_scores = []
    df1 = df.copy()
    df1 = df[cols]
    
    # 5 Fold cross validation stratified on the basis of target
    kf = StratifiedKFold(n_splits=5,random_state=rstate,shuffle=True)
    for df_index,test_index in kf.split(df1,y_all):
        print('\n{} of kfold {}'.format(i,kf.n_splits))
        xtr,xvl = df1.loc[df_index],df1.loc[test_index]
        ytr,yvl = y_all.loc[df_index],y_all.loc[test_index]
            
        # Define model for fitting on the training set for each fold
        model = ml_model
        model.fit(xtr, ytr)
        pred_probs = model.predict_proba(xvl)
        pp = []
         
        # Use threshold to define the classes based on probability values
        for j in pred_probs[:,1]:
            if j>thres:
                pp.append(1)
            else:
                pp.append(0)
         
        # Calculate scores for each fold and print
        pred_val = pp
        roc_score = roc_auc_score(yvl,pred_probs[:,1])
        recall = recall_score(yvl,pred_val)
        precision = precision_score(yvl,pred_val)
        sufix = ""
        msg = ""
        msg += "ROC AUC Score: {}, Recall Score: {:.4f}, Precision Score: {:.4f} ".format(roc_score, recall,precision)
        print("{}".format(msg))
         
         # Save scores
        cv_scores.append(roc_score)
        i+=1
    return cv_scores

In [ ]:
baseline_scores = cv_score(LogisticRegression(), cols = baseline_cols)

In [ ]:
all_feat_scores = cv_score(LogisticRegression())

There is some improvement in both ROC AUC Scores and Precision/Recall Scores. Now we can try backward selection to select the best subset of features which give the best score. 

### Reverse Feature Elimination or Backward Selection

We have already built a model using all the features and a separate model using some baseline features. We can try using backward feature elimination to check if we can do better. Let's do that next.

In [ ]:
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt

# Create the RFE object and rank each feature
model = LogisticRegression()
rfe = RFE(estimator=model, n_features_to_select=1, step=1)
rfe.fit(, Y)

In [ ]:
#make the ranking of the variables 
ranking_df = pd.DataFrame()
ranking_df['Feature_name'] = X.columns
ranking_df['Rank'] = rfe.ranking_

In [ ]:
ranked = ranking_df.sort_values(by=['Rank']
ranked

In [ ]:
rfe_top_10_scores = cv_score(LogisticRegression(), cols = ranked['Feature_name'][:10].values)

In [ ]:
cv_score(LogisticRegression(), cols = ranked['Feature_name'][:10].values, thres=0.14)

## Comparison of Different model fold wise

Let us visualise the cross validation scores for each fold for the following 3 models and observe differences:
* Baseline Model
* Model based on all features
* Model based on top 10 features obtained from RFE

In [ ]:
results_df = pd.DataFrame({'baseline':baseline_scores, 'all_feats': all_feat_scores, 'rfe_top_10': rfe_top_10_scores})

In [ ]:
results_df.plot(y=["baseline", "all_feats", "rfe_top_10"], kind="bar")